In [ ]:
from openai import OpenAI
import pandas as pd
import os
import tqdm
import time
from datasets import load_dataset

OPENAI_API_KEY = '-'

In [12]:
dataset = load_dataset("jiacheng-ye/nl2bash", split="train")

df = pd.DataFrame(dataset)

df.to_csv("nl2bash_full.csv", index=False, encoding="utf-8-sig")
print(f"저장 완료: {len(df)}개 명령어 → nl2bash_full.csv")

README.md:   0%|          | 0.00/119 [00:00<?, ?B/s]

nl2bash.py:   0%|          | 0.00/2.24k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/435k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/40.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8090 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/609 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/606 [00:00<?, ? examples/s]

저장 완료: 8090개 명령어 → nl2bash_full.csv


In [2]:
df = pd.read_csv("nl2bash_full.csv")
english_nls = df["nl"].tolist()
bash_cmds = df["bash"].tolist()

In [3]:
client = OpenAI(api_key=OPENAI_API_KEY)

def translate_batch(texts):
    prompt = "다음 명령어 설명을 자연스럽고 정확한 한국어로 번역해줘:\n\n"
    for i, t in enumerate(texts):
        prompt += f"{i+1}. {t}\n"

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response.choices[0].message.content

paired_results = []
batch_size = 100   
sleep_sec = 0.8   

for i in range(0, len(english_nls), batch_size):
    batch_nls = english_nls[i:i+batch_size]
    batch_bash = bash_cmds[i:i+batch_size]

    print(f"번역 중 {i} ~ {i + len(batch_nls) - 1}")

    try:
        result = translate_batch(batch_nls)
        lines = [line.split(".", 1)[1].strip() for line in result.strip().split("\n") if "." in line]

        # 누락 보정
        if len(lines) < len(batch_nls):
            print(f"누락 감지: {len(lines)} / {len(batch_nls)}")
            lines += ["__MISSING__"] * (len(batch_nls) - len(lines))

        paired_results.extend(zip(lines, batch_bash))

    except Exception as e:
        print(f"오류 발생: {e}")
        paired_results.extend(zip(["__MISSING__"] * len(batch_nls), batch_bash))

    time.sleep(sleep_sec) 

번역 중 0 ~ 99
번역 중 100 ~ 199
번역 중 200 ~ 299
번역 중 300 ~ 399
번역 중 400 ~ 499
번역 중 500 ~ 599
번역 중 600 ~ 699
번역 중 700 ~ 799
번역 중 800 ~ 899
번역 중 900 ~ 999
번역 중 1000 ~ 1099
번역 중 1100 ~ 1199
번역 중 1200 ~ 1299
번역 중 1300 ~ 1399
번역 중 1400 ~ 1499
번역 중 1500 ~ 1599
누락 감지: 99 / 100
번역 중 1600 ~ 1699
번역 중 1700 ~ 1799
번역 중 1800 ~ 1899
번역 중 1900 ~ 1999
번역 중 2000 ~ 2099
번역 중 2100 ~ 2199
번역 중 2200 ~ 2299
번역 중 2300 ~ 2399
번역 중 2400 ~ 2499
번역 중 2500 ~ 2599
번역 중 2600 ~ 2699
번역 중 2700 ~ 2799
번역 중 2800 ~ 2899
번역 중 2900 ~ 2999
번역 중 3000 ~ 3099
번역 중 3100 ~ 3199
번역 중 3200 ~ 3299
번역 중 3300 ~ 3399
번역 중 3400 ~ 3499
번역 중 3500 ~ 3599
번역 중 3600 ~ 3699
번역 중 3700 ~ 3799
번역 중 3800 ~ 3899
번역 중 3900 ~ 3999
번역 중 4000 ~ 4099
번역 중 4100 ~ 4199
번역 중 4200 ~ 4299
번역 중 4300 ~ 4399
번역 중 4400 ~ 4499
번역 중 4500 ~ 4599
번역 중 4600 ~ 4699
번역 중 4700 ~ 4799
번역 중 4800 ~ 4899
번역 중 4900 ~ 4999
번역 중 5000 ~ 5099
번역 중 5100 ~ 5199
번역 중 5200 ~ 5299
번역 중 5300 ~ 5399
번역 중 5400 ~ 5499
번역 중 5500 ~ 5599
번역 중 5600 ~ 5699
번역 중 5700 ~ 5799
번역 중 5800 ~ 5899
번역 중

In [ ]:
# 저장
df_translated = pd.DataFrame(paired_results, columns=["instruction", "output"])
df_translated.to_csv("./dataset/korean_nl2bash.csv", index=False, encoding="utf-8-sig")

print("저장 완료: korean_nl2bash.csv")

저장 완료: korean_nl2bash.csv


In [ ]:
from datasets import Dataset, DatasetDict
from huggingface_hub import HfApi, HfFolder

csv_path = "korean_nl2bash.csv"

dataset = Dataset.from_csv(f'./dataset/{csv_path}')

dataset_dict = DatasetDict({
    "train": dataset
})

repo_id = "rlawltjd/korean-nl2bash"

dataset_dict.push_to_hub(repo_id)


Generating train split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/rlawltjd/korean-nl2bash/commit/02dd36b0da90df69b664c7a2bb6157e5bea46b5d', commit_message='Upload dataset', commit_description='', oid='02dd36b0da90df69b664c7a2bb6157e5bea46b5d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/rlawltjd/korean-nl2bash', endpoint='https://huggingface.co', repo_type='dataset', repo_id='rlawltjd/korean-nl2bash'), pr_revision=None, pr_num=None)